In [1]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download -d thoughtvector/customer-support-on-twitter

 91% 153M/169M [00:01<00:00, 142MB/s]
100% 169M/169M [00:01<00:00, 133MB/s]


In [3]:
!unzip /content/customer-support-on-twitter.zip

Archive:  /content/customer-support-on-twitter.zip
  inflating: sample.csv              
  inflating: twcs/twcs.csv           


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [6]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [7]:
df = pd.read_csv('/content/sample.csv')

In [8]:
for index, row in df.iterrows():
    print(row['text'])

@AppleSupport causing the reply to be disregarded and the tapped notification under the keyboard is opened😡😡😡
@105835 Your business means a lot to us. Please DM your name, zip code and additional details about your concern. ^RR https://t.co/znUu1VJn9r
@76328 I really hope you all change but I'm sure you won't! Because you don't have to!
@105836 LiveChat is online at the moment - https://t.co/SY94VtU8Kq or contact 03331 031 031 option 1, 4, 3 (Leave a message) to request a call back
@VirginTrains see attached error message. I've tried leaving a voicemail several times in the past week https://t.co/NxVZjlYx1k
@105836 Have you tried from another device, Miriam ^MM
@VirginTrains yep, I've tried laptop too several times over the past week and again today. I've tried different browsers too
@105836 It's working OK from here, Miriam. Does this link help https://t.co/0m2mpH15eh ? ^MM
@VirginTrains I still haven't heard &amp; the number I'm directed to by phone is a dead end &amp; the live chat 

In [9]:
# supprimer la ponctuation
df['text'] = df['text'].str.replace('[^\w\s]', '')
for index, row in df.head(10).iterrows():
    print(row['text'])

AppleSupport causing the reply to be disregarded and the tapped notification under the keyboard is opened
105835 Your business means a lot to us Please DM your name zip code and additional details about your concern RR httpstcoznUu1VJn9r
76328 I really hope you all change but Im sure you wont Because you dont have to
105836 LiveChat is online at the moment  httpstcoSY94VtU8Kq or contact 03331 031 031 option 1 4 3 Leave a message to request a call back
VirginTrains see attached error message Ive tried leaving a voicemail several times in the past week httpstcoNxVZjlYx1k
105836 Have you tried from another device Miriam MM
VirginTrains yep Ive tried laptop too several times over the past week and again today Ive tried different browsers too
105836 Its working OK from here Miriam Does this link help httpstco0m2mpH15eh  MM
VirginTrains I still havent heard amp the number Im directed to by phone is a dead end amp the live chat doesnt work Can someone call me
105836 Thats what were here for M

<ipython-input-9-72119ddb44aa>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('[^\w\s]', '')


In [13]:
df['text'] = df['text'].str.lower()
print("_____Apres la conversion en minuscules:_____")
for index, row in df.head(10).iterrows():
    print(row['text'])


_____Apres la conversion en minuscules:_____
applesupport causing the reply to be disregarded and the tapped notification under the keyboard is opened
105835 your business means a lot to us please dm your name zip code and additional details about your concern rr httpstcoznuu1vjn9r
76328 i really hope you all change but im sure you wont because you dont have to
105836 livechat is online at the moment  httpstcosy94vtu8kq or contact 03331 031 031 option 1 4 3 leave a message to request a call back
virgintrains see attached error message ive tried leaving a voicemail several times in the past week httpstconxvzjlyx1k
105836 have you tried from another device miriam mm
virgintrains yep ive tried laptop too several times over the past week and again today ive tried different browsers too
105836 its working ok from here miriam does this link help httpstco0m2mph15eh  mm
virgintrains i still havent heard amp the number im directed to by phone is a dead end amp the live chat doesnt work can some

In [14]:
print("_____Après la suppression des espaces supplémentaires:_____")
df['text'] = df['text'].str.strip()
for index, row in df.head(10).iterrows():
    print(row['text'])

_____Après la suppression des espaces supplémentaires:_____
applesupport causing the reply to be disregarded and the tapped notification under the keyboard is opened
105835 your business means a lot to us please dm your name zip code and additional details about your concern rr httpstcoznuu1vjn9r
76328 i really hope you all change but im sure you wont because you dont have to
105836 livechat is online at the moment  httpstcosy94vtu8kq or contact 03331 031 031 option 1 4 3 leave a message to request a call back
virgintrains see attached error message ive tried leaving a voicemail several times in the past week httpstconxvzjlyx1k
105836 have you tried from another device miriam mm
virgintrains yep ive tried laptop too several times over the past week and again today ive tried different browsers too
105836 its working ok from here miriam does this link help httpstco0m2mph15eh  mm
virgintrains i still havent heard amp the number im directed to by phone is a dead end amp the live chat doesn

In [15]:
# 4. Supprimer les stopwords
stop_words = set(stopwords.words('english'))
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
print("_____Après la suppression des stopwords:_____")
for index, row in df.head(10).iterrows():
    print(row['text'])

_____Après la suppression des stopwords:_____
applesupport causing reply disregarded tapped notification keyboard opened
105835 business means lot us please dm name zip code additional details concern rr httpstcoznuu1vjn9r
76328 really hope change im sure wont dont
105836 livechat online moment httpstcosy94vtu8kq contact 03331 031 031 option 1 4 3 leave message request call back
virgintrains see attached error message ive tried leaving voicemail several times past week httpstconxvzjlyx1k
105836 tried another device miriam mm
virgintrains yep ive tried laptop several times past week today ive tried different browsers
105836 working ok miriam link help httpstco0m2mph15eh mm
virgintrains still havent heard amp number im directed phone dead end amp live chat doesnt work someone call
105836 thats miriam team send email shortly hp


In [16]:
# Remove numbers
df['text'] = df['text'].str.replace('\d', '')
for index, row in df.head(10).iterrows():
    print(row['text'])

applesupport causing reply disregarded tapped notification keyboard opened
 business means lot us please dm name zip code additional details concern rr httpstcoznuuvjnr
 really hope change im sure wont dont
 livechat online moment httpstcosyvtukq contact    option    leave message request call back
virgintrains see attached error message ive tried leaving voicemail several times past week httpstconxvzjlyxk
 tried another device miriam mm
virgintrains yep ive tried laptop several times past week today ive tried different browsers
 working ok miriam link help httpstcommpheh mm
virgintrains still havent heard amp number im directed phone dead end amp live chat doesnt work someone call
 thats miriam team send email shortly hp


<ipython-input-16-83c6855ddb50>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('\d', '')


In [17]:
#Tokenization
df['text'] = df['text'].apply(word_tokenize)
for index, row in df.head(10).iterrows():
    print(row['text'])

['applesupport', 'causing', 'reply', 'disregarded', 'tapped', 'notification', 'keyboard', 'opened']
['business', 'means', 'lot', 'us', 'please', 'dm', 'name', 'zip', 'code', 'additional', 'details', 'concern', 'rr', 'httpstcoznuuvjnr']
['really', 'hope', 'change', 'im', 'sure', 'wont', 'dont']
['livechat', 'online', 'moment', 'httpstcosyvtukq', 'contact', 'option', 'leave', 'message', 'request', 'call', 'back']
['virgintrains', 'see', 'attached', 'error', 'message', 'ive', 'tried', 'leaving', 'voicemail', 'several', 'times', 'past', 'week', 'httpstconxvzjlyxk']
['tried', 'another', 'device', 'miriam', 'mm']
['virgintrains', 'yep', 'ive', 'tried', 'laptop', 'several', 'times', 'past', 'week', 'today', 'ive', 'tried', 'different', 'browsers']
['working', 'ok', 'miriam', 'link', 'help', 'httpstcommpheh', 'mm']
['virgintrains', 'still', 'havent', 'heard', 'amp', 'number', 'im', 'directed', 'phone', 'dead', 'end', 'amp', 'live', 'chat', 'doesnt', 'work', 'someone', 'call']
['thats', 'miriam

In [19]:
# Lemmatization
lemmatizer = WordNetLemmatizer()
def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

df['text'] = df['text'].apply(lemmatize_tokens)
for index, row in df.head(10).iterrows():
    print(row['text'])

['applesupport', 'causing', 'reply', 'disregarded', 'tapped', 'notification', 'keyboard', 'opened']
['business', 'mean', 'lot', 'u', 'please', 'dm', 'name', 'zip', 'code', 'additional', 'detail', 'concern', 'rr', 'httpstcoznuuvjnr']
['really', 'hope', 'change', 'im', 'sure', 'wont', 'dont']
['livechat', 'online', 'moment', 'httpstcosyvtukq', 'contact', 'option', 'leave', 'message', 'request', 'call', 'back']
['virgintrains', 'see', 'attached', 'error', 'message', 'ive', 'tried', 'leaving', 'voicemail', 'several', 'time', 'past', 'week', 'httpstconxvzjlyxk']
['tried', 'another', 'device', 'miriam', 'mm']
['virgintrains', 'yep', 'ive', 'tried', 'laptop', 'several', 'time', 'past', 'week', 'today', 'ive', 'tried', 'different', 'browser']
['working', 'ok', 'miriam', 'link', 'help', 'httpstcommpheh', 'mm']
['virgintrains', 'still', 'havent', 'heard', 'amp', 'number', 'im', 'directed', 'phone', 'dead', 'end', 'amp', 'live', 'chat', 'doesnt', 'work', 'someone', 'call']
['thats', 'miriam', 'te

In [20]:
# Join the lemmatized tokens to form paragraphs
df['paragraph'] = df['text'].apply(lambda tokens: ' '.join(tokens))

# Print the paragraphs
for paragraph in df['paragraph']:
    print(paragraph)

applesupport causing reply disregarded tapped notification keyboard opened
business mean lot u please dm name zip code additional detail concern rr httpstcoznuuvjnr
really hope change im sure wont dont
livechat online moment httpstcosyvtukq contact option leave message request call back
virgintrains see attached error message ive tried leaving voicemail several time past week httpstconxvzjlyxk
tried another device miriam mm
virgintrains yep ive tried laptop several time past week today ive tried different browser
working ok miriam link help httpstcommpheh mm
virgintrains still havent heard amp number im directed phone dead end amp live chat doesnt work someone call
thats miriam team send email shortly hp
help version io find setting gt general gt reply dm httpstcogdrquypt
applesupport suffering hope find solution
applesupport hi apple ive concern latest io slow iphone happy solution please
thanks reaching u always happy help send u dm look together httpstcogdrquypt
updated phone sudden